# Optimal Execution Strategy with Price Limiter (Chap 7.2) Helper Function File
This file contains the PDE solver function and all plotting functionality in the main file.

In [1]:
# Importing Packges and defining plot parameters
# Importing Packges
import time
import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from scipy import interpolate
from mpl_toolkits.mplot3d import Axes3D
np.random.seed(20)  # Setting random seed
np.seterr(divide='ignore', invalid='ignore')
import matplotlib.pylab as pylab
params = {'legend.fontsize': 10,
          'figure.figsize': (8, 4),
         'axes.labelsize': 20,
         'axes.titlesize': 20,
         'xtick.labelsize': 15,
         'ytick.labelsize': 15}
pylab.rcParams.update(params)
font = {'family': 'serif',
        'style': 'italic',
        'weight': 1,
        'size': 16,
        }

The following function solves the PDE for $h(t,S)$, which is used in the optimal execution strategy, using a Crank-Nicholson scheme. 

In [4]:
def solve_pde_hjb(Fmin, Fmax, NdF, alpha, phi, k, sigma, T, Ndt):
    # Initialization
    dt = T/Ndt  # Time change increment
    dF = (Fmax - Fmin)/(NdF - 1)
    h = np.full([NdF, Ndt+1], np.nan)  # Solution

    h[:, -1] = alpha  # Boundary/Terminal Condition
    ainv = 1/k
    dtanddF = 0.25*dt/(dF**2) # coefficient that shows up in the FD method

    # the M matrix in the Crank-Nicholson scheme
    M = np.zeros([NdF-2, NdF-2])
    M[0, 0] = 1
    M[NdF-3, NdF-3] = 1 + 2*dtanddF*sigma**2
    M[NdF-3, NdF-4] = -dtanddF*sigma**2

    for i in range(2, NdF-2):
        i_0 = i-1
        M[i_0, i_0-1] = - dtanddF*sigma**2
        M[i_0, i_0] = 1 + 2*dtanddF*sigma**2
        M[i_0, i_0+1] = - dtanddF*sigma**2
    Minv = np.linalg.inv(M)
    
    
    for i in range(Ndt-1, -1, -1):

        H = h[:, i+1].reshape(h.shape[0], 1)
        E = np.full([NdF, 1], np.nan)
        E[1:NdF-1] = H[1:NdF-1] \
        + dtanddF*(sigma**2)*(H[2:NdF] - 2*H[1:NdF-1] + H[:NdF-2]) \
        - dt*ainv*np.power(H[1:NdF-1], 2) + dt*phi
        
        v = E[1: NdF-1]

        v[NdF-3] = v[NdF-3] + dtanddF * sigma**2 * alpha

        h[1:NdF-1, i] = (Minv@v).reshape(NdF-2,)

        h[NdF-1, i] = alpha
        h[0, i] = 2*h[1, i] - h[2, i]

    t = np.arange(0, T+10**-9, dt)
    F = np.arange(Fmin, Fmax+10**-9, dF)
    return h, F, t

The following function plots the rate of aquisition as a functin of time and fundamental price base on the approximated HJB equation solution.

In [1]:
def plot_stategy(h, t, Fgrid, F0, T, Fmin, Fmax):
    
    fig = plt.figure()
    ax = plt.gca(projection='3d')
    axes = fig.gca()
    axes.set_xlim([0, T])
    axes.set_ylim([F0, Fmax])
    indx = Fgrid >= F0
    Z_plot = h[indx, :]
    x = t
    y = Fgrid[indx]
    X_plot, Y_plot = np.meshgrid(x, y)

    # Scale each axis for visualization
    x_scale = 1
    y_scale = 1
    z_scale = 2/3
    scale = np.diag([x_scale, y_scale, z_scale, 1.0])
    scale = scale * (1.0 / scale.max())
    scale[3, 3] = 1.0
    
    # Helper function for scaling each axis on plot
    def short_proj():
        return np.dot(Axes3D.get_proj(ax), scale)
    ax.get_proj = short_proj

    surf = ax.plot_surface(X_plot, Y_plot, Z_plot, cmap='viridis', linewidth=0, antialiased=False)
    fig.colorbar(surf, shrink=0.5, aspect=5)
    ax.set_xlabel('t', fontsize=10)
    ax.set_xticks([0, 0.5, 1])
    ax.set_ylabel('S', fontsize=10)
    ax.set_yticks([20, 20.05, 20.1])
    ax.set_zlabel(r'$h(t,S)$', fontsize=10)
    ax.set_zticks([0, 50, 100])
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.view_init(30, 225)
    plt.tight_layout()
    plt.subplots_adjust(left=0.001, bottom=0.001)

In [4]:
# Plot Path for Price VS Time
def PlotPricePathMap(T, t, F, Fmin, Fmax, S, idxfig, lw):
    fig_1 = plt.figure()
    axes = fig_1.gca()
    axes.set_xlim((0, T))
    axes.set_ylim([Fmin-0.05, Fmax+0.05])
    plt.tick_params(direction='in', bottom=True, top=True, left=True, right=True)

    F[S==0] = np.nan
    for i in range(len(idxfig)):
        plt.plot(t, F[idxfig[i]], linewidth=lw, label=i+1)
    plt.hlines(Fmax, 0, T, linestyles='dotted')
    plt.legend(['1', '2', '3'], loc='lower right')
    plt.ylabel('Fundamental Price ' + r'$(S_t)$',  fontdict=font)
    plt.xlabel('Time ' + r'$(T)$',  fontdict=font)

In [5]:
# Plot Path for Inventory VS Time
def PlotInvPathMap(t, Q_AC, Q, idxfig, lw):
    fig_2 = plt.figure()
    axes = fig_2.gca()
    axes.set_xlim(left=0)
    axes.set_ylim(bottom=0)
    plt.tick_params(direction='in', bottom=True, top=True, left=True, right=True)

    plt.plot(t, Q_AC[0, :], linestyle='dotted', linewidth=3, label='AC')
    for i in range(len(idxfig)):
        plt.plot(t, Q[idxfig[i], :], linewidth=lw, label=i+1)

    plt.legend()
    plt.ylabel('Inventory ' +r'$(Q_t)$', fontdict=font)
    plt.xlabel('Time ' + r'$(T)$', fontdict=font)

In [6]:
# Plot Path for Trade Speed VS Time
def PlotTradeSpeedPathMap(T, t, nu_AC, nu, idxfig, lw):
    fig_3 = plt.figure()
    axes = fig_3.gca()
    axes.set_xlim((0, T))
    axes.set_ylim([0, max(nu_AC[0,:])*1.25])
    plt.tick_params(direction='in', bottom=True, top=True, left=True, right=True)

    plt.plot(t, nu_AC[0, :], linestyle='dotted', linewidth=3, label='AC')
    for i in range(len(idxfig)):
        plt.plot(t, nu[idxfig[i]], linewidth=lw, label=i+1)

    plt.ylabel('Trading Speed ' +r'$(v_t)$', fontdict=font)
    plt.xlabel('Time ' + r'$(T)$', fontdict=font)
    plt.legend()


In [7]:
# Plot Path for Cost per Share VS Time
def PlotCostPathMap(t, X, Q, F0, Fmax, idxfig, lw):
    fig_4 = plt.figure()
    axes = fig_4.gca()
    axes.set_xlim(left=0)
    axes.set_ylim([F0*0.99, Fmax])
    plt.tick_params(direction='in', bottom=True, top=True, left=True, right=True)
    for i in range(len(idxfig)):
        plt.plot(t, np.divide(X[idxfig[i]], Q[idxfig[i], :]), linewidth=lw, label=i+1)
    plt.ylabel('Cost per Share', fontdict=font)
    plt.xlabel('Time ' + r'$(T)$', fontdict=font)
    plt.legend()


In [8]:
# Plot Heat Map for Inventory VS Time
def PlotInvHeatMap(t, Q, Q_AC, lower_cutoff, Nsims):
    fig_5 = plt.figure()
    plt.tick_params(direction='in', bottom=True, top=True, left=True, right=True)

    count_mat_i = np.full([np.arange(0, 1, 0.01).shape[0], len(t)], np.nan)
    for i in range(len(t)):
        count_mat_i[:, i] = np.histogram(Q[:, i], bins=np.arange(0, 1.001, 0.01))[0]
    x_cord_i, y_cord_i = np.meshgrid(t, np.arange(0, 1, 0.01))
    count_mat_i[count_mat_i <= lower_cutoff] = 0
    z = count_mat_i/Nsims
    cmap = plt.get_cmap('YlOrRd')
    plt.contour(x_cord_i, y_cord_i, z, 100, cmap=cmap, levels=np.linspace(z.min(), z.max(), 1000))
    plt.colorbar(ticks=np.arange(0, 1, 0.1))

    plt.plot(t, Q.mean(axis=0), linewidth=2, linestyle='-', color='blue', label = 'Optimal Inv')
    plt.plot(t, Q_AC.mean(axis=0), linewidth=2, linestyle='--', color='black', label = 'AC Inv')
    plt.xlim(0, 1)
    plt.ylim(0, 1)

    plt.ylabel('Inventory ' +r'$(Q_t)$', fontdict=font)
    plt.xlabel('Time ' + r'$(T)$', fontdict=font)
    plt.legend()



In [9]:
# Plot Heat Map for Trading Speed VS Time
def PlotTradeSpeedHeatMap(t, nu, nu_AC, lower_cutoff, Nsims):
    fig_6 = plt.figure()
    plt.tick_params(direction='in', bottom=True, top=True, left=True, right=True)

    count_mat_ts = np.full([np.arange(0, 5, 0.025).shape[0], len(t)], np.nan)

    for i in range(len(t)):
        count_mat_ts[:, i] = np.histogram(nu[:, i], bins=np.arange(0, 5+0.0001, 0.025))[0]
    x_cord_ts, y_cord_ts = np.meshgrid(t, np.arange(0, 5, 0.025))
    count_mat_ts[count_mat_ts <= lower_cutoff] = 0
    z_ts = count_mat_ts/Nsims
    cmap = plt.get_cmap('YlOrRd')
    plt.contour(x_cord_ts, y_cord_ts, z_ts, 100, cmap=cmap, levels=np.linspace(z_ts.min(), z_ts.max(), 1000))
    plt.colorbar(ticks=np.arange(0, 1, 0.1))

    plt.plot(t, nu.mean(axis=0), '-b', linewidth=2, label = 'Optimal Average Trade Speed')
    plt.plot(t, nu_AC[0, :], '--k', linewidth=2, label = 'AC Average Trade Speed')
    plt.ylabel('Trading Speed ' +r'$(v_t)$', fontdict=font)
    plt.xlabel('Time ' + r'$(T)$', fontdict=font)
    plt.legend()


In [4]:
# Plot Histagram for Price
def PlotPriceHist(X, Q):
    fig_6 = plt.figure()
    axes = fig_6.gca()
    data = np.divide(X[:, -1], Q[:, -1])
    axes.set_xlim(xmin = np.percentile(data, 1), xmax = np.percentile(data, 99))
    plt.tick_params(direction='in', bottom=True, top=True, left=True, right=True)

    upper = np.percentile(data, 99)
    lower = np.percentile(data, 1)
    plt.hist(data, np.arange(lower, upper, (upper-lower)/50))
    plt.tick_params(axis="x", width=(upper-lower)/5)

    plt.ylabel('Frequency', fontdict=font)
    plt.xlabel('Price', fontdict=font)    
    

In [1]:
# Plot Histagram for Price of AC Strategy
def PlotPriceACHist(X_AC, Q_AC):
    fig_7 = plt.figure()
    data = np.divide(X_AC[:, -1], Q_AC[:, -1])
    axes = fig_7.gca()
    axes.set_xlim(xmin = np.percentile(data, 1), xmax = np.percentile(data, 99))
    plt.tick_params(direction='in', bottom=True, top=True, left=True, right=True)
    
    upper = np.percentile(data, 99)
    lower = np.percentile(data, 1)
    plt.hist(data, np.arange(lower, upper, (upper-lower)/50))
    plt.tick_params(axis="x", width=(upper-lower)/5)

    plt.ylabel('Frequency', fontdict=font)
    plt.xlabel('Price_AC', fontdict=font)

In [2]:
# Plot Histagram for Price Difference between optimal and AC Strategy
def PlotPriceDifHist(X, X_AC):
    fig_9 = plt.figure()
    data = np.subtract(X[:, -1], X_AC[:, -1])
    axes = fig_9.gca()
    axes.set_xlim(xmin = np.percentile(data, 1), xmax = np.percentile(data, 99))
    
    upper = np.percentile(data, 99)
    lower = np.percentile(data, 1)
    plt.hist(data, np.arange(lower, upper, (upper-lower)/50))
    plt.tick_params(axis="x", width=(upper-lower)/5)
 
    plt.ylabel('Frequency', fontdict=font)
    plt.xlabel('Price_Diff', fontdict=font)


In [3]:
# Plot Histagram for I (Name to be finalized)
def PlotIHist(I):
    fig_11 = plt.figure()
    axes = fig_11.gca()
    axes.set_xlim(xmin = np.percentile(I, 1), xmax = np.percentile(I, 99))
    plt.tick_params(direction='in', bottom=True, top=True, left=True, right=True)
    
    upper = np.percentile(I, 99)
    lower = np.percentile(I, 1)
    plt.hist(I, np.arange(lower, upper, (upper-lower)/50))
    plt.tick_params(axis="x", width=(upper-lower)/5)

    plt.ylabel('Frequency', fontdict=font)
    plt.xlabel('Cost', fontdict=font)
